In [1]:
import sys, os
import numpy as np
import pandas as pd
from geopandas import GeoDataFrame
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pdb
from joblib import Parallel, delayed
import multiprocessing
from scipy.stats import entropy
from datetime import datetime, timedelta
import epiweeks as epi
from scipy.stats import pearsonr
from scipy.signal import correlate
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.fftpack import fft, ifft
from scipy.optimize import nnls
import scipy.stats as stats
from scipy.optimize import leastsq
from scipy.signal import savgol_filter
from collections import defaultdict
import statsmodels.api as sm
from tqdm import tqdm
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

In [2]:
def get_week(date, weeks):
    for week in weeks:
        s,e = week.split('_')
        if s <= date and date <= e:
            return week

fipsdf=pd.read_csv('../misc_data/US_fips_codes_names.csv')
fipsdf['FIPS']=fipsdf.FIPS.apply(lambda x: '{:05}'.format(x))
mapfips=dict(zip(fipsdf['FIPS'],fipsdf['County']+'_'+fipsdf['name']))
mapname=dict(zip(fipsdf['County']+'_'+fipsdf['name'],fipsdf['FIPS']))
dictfips=fipsdf[['FIPS','State_code']].rename(columns={'FIPS':'cnty'})
dictfips['st_fips_code']=dictfips['cnty'].str[0:2]

goog_dates=[]
with open('input/goog_dates.txt','r') as f:
    for line in f:
        goog_dates.append(line[:-1])
goog_dates
stdt=goog_dates[-1].split('_')[0]
for i in range(1,20):
    dt0=(datetime.strptime(stdt,'%Y-%m-%d')+timedelta(weeks=i)).strftime('%Y-%m-%d')
    dt1=(datetime.strptime(dt0,'%Y-%m-%d')+timedelta(days=6)).strftime('%Y-%m-%d')
    goog_dates.append(dt0+'_'+dt1)

sorted(goog_dates)[-1]

'2023-01-29_2023-02-04'

In [3]:
file="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
df = pd.read_csv(file,header=0,dtype={'UID':str,'code3':str,'FIPS':str})
df=df.dropna(subset=['FIPS'])
df.loc[:,'FIPS']=df.FIPS.apply(lambda x : '{:05}'.format(int(float(x))))
all_dict={}
for st in df.Province_State:
        all_dict[st]=dict(zip(df[df.Province_State==st]['FIPS'],df[df.Province_State==st]['Admin2']))
df.drop(columns=['Country_Region','Lat','Long_','iso2','iso3','UID','code3','Admin2','Combined_Key','Province_State'],inplace=True)
agg_df=df.dropna(subset=['FIPS'])
agg_df.loc[:,'FIPS']=agg_df.FIPS.astype(float)
agg_df.FIPS=agg_df.FIPS.apply(lambda x: '{:05}'.format(int(x)))
# agg_df = df.groupby(['Province_State']).agg(np.sum)
dates = agg_df.iloc[:,1:].columns.tolist()
dates = [datetime.strptime(x, '%m/%d/%y') for x in dates]
dates = [x.strftime('%Y-%m-%d') for x in dates]
agg_df.iloc[:,1:].columns = dates
for x in agg_df.iloc[:,1:].columns:
    agg_df=agg_df.rename(columns={x:datetime.strptime(x, '%m/%d/%y').strftime('%Y-%m-%d')})
# agg_df.loc[:,'admin_st']=agg_df.Admin2+'_'+agg_df.Province_State
# agg_df=agg_df.drop(columns=['Admin2','Province_State'])
agg_df=agg_df.set_index('FIPS')
# dates = agg_df.columns
agg_df.groupby((np.arange(len(agg_df.columns)) // 7) + 1, axis=1).sum().add_prefix('s')
agg_df.columns = [get_week(x,goog_dates) for x in dates]
agg_df = agg_df.groupby(agg_df.columns,axis=1).max()
dates = agg_df.columns.values.tolist()
new_df = agg_df.copy()
for i in range(len(dates)):
    d, prev_d = dates[i], dates[i-1]
    new_df[d] = agg_df[d] - agg_df[prev_d]
    
covid_df = new_df.copy()
covid_dates = dates

# agg_df.index=agg_df.index.map(mapname)
# agg_df=agg_df[agg_df.index.notna()]
# new_df.index=new_df.index.map(mapname)
# new_df=new_df[new_df.index.notna()]

for col in agg_df.columns:
    agg_df=agg_df.rename(columns={col:col.split('_')[0]})
for col in new_df.columns:
    new_df=new_df.rename(columns={col:col.split('_')[0]})
    

for col in new_df.columns:
    new_df=new_df.rename(columns={col:datetime.strptime(col,'%Y-%m-%d').date()})

diff_df=new_df.diff(axis=1).fillna(0)
data_df=np.log(new_df.mask(new_df<=0)).fillna(0)#diff_df

new_df[new_df<=0]=1

for col in new_df.columns:
    if col=='FIPS':
        continue
    else:
        new_df=new_df.rename(columns={col:col.strftime('%Y-%m-%d')})
newdf=new_df.reset_index()
cols=new_df.columns
mnewdf=pd.melt(newdf,id_vars=['FIPS'], value_vars=cols,
        var_name='fct_date', value_name='true')
mnewdf=mnewdf.rename(columns={'FIPS':'cnty'})

In [4]:
popdf=pd.read_csv('../misc_data1/countypops_2016.txt',sep=" ", header=None, names=['cnty','pop'], dtype={'cnty':str})

In [5]:
tempdf=pd.read_csv('output/alldates_allmethods_with_weights_2020-11-08_v3.csv')

In [6]:
# tempdf=tempdf[tempdf.horizon>='2020-05-31']
chdf=pd.read_csv('https://raw.githubusercontent.com/reichlab/covid19-forecast-hub/master/data-processed/COVIDhub-ensemble/2021-01-18-COVIDhub-ensemble.csv')
locs=list(chdf.location.unique())

In [7]:
hrzndt='2021-04-11'

In [8]:
tdf=pd.read_csv('output/bma_all_methods_PatchSim_adpt-B117_{}.csv'.format(hrzndt))


In [9]:
tdf.cnty=pd.to_numeric(tdf.cnty)
tdf.cnty=tdf.cnty.apply(lambda x: '{:05}'.format(int(x)))
tdf=tdf[tdf.method=='BMA-ensemble']
tdf.loc[:,'fct_lb']=tdf.loc[:,'fct_mean']-1.96*tdf.loc[:,'fct_std']
tdf.loc[:,'fct_ub']=tdf.loc[:,'fct_mean']+1.96*tdf.loc[:,'fct_std']

In [10]:
col=['cnty', 'step_ahead', 'horizon', 'fct_date', 'fct_mean', 'fct_lb', 'fct_ub']

In [11]:
infile='output/bma_all_methods_PatchSim_adpt-B117_{}.csv'.format(hrzndt)

In [12]:
locs=list(chdf.location.unique())
ll=[]
holieff=0
for dt in [hrzndt]:#tempdf.horizon.unique():
    sdt=(datetime.strptime(dt,'%Y-%m-%d')+timedelta(days=8)).strftime('%Y-%m-%d')
#     try:
    tdf=pd.read_csv(infile,dtype={'cnty':str})
    tdf.cnty=pd.to_numeric(tdf.cnty)
    tdf.cnty=tdf.cnty.apply(lambda x: '{:05}'.format(int(x)))
    tdf=tdf[tdf.method=='BMA-ensemble']
    tdf.loc[:,'fct_lb']=tdf.loc[:,'fct_mean']-1.96*tdf.loc[:,'fct_std']
    tdf.loc[:,'fct_ub']=tdf.loc[:,'fct_mean']+1.96*tdf.loc[:,'fct_std']
#     tdf[tdf.cnty!='US'].cnty=pd.to_numeric(tdf[tdf.cnty!='US'].cnty)
#     tdf[tdf.cnty!='US'].cnty=tdf[tdf.cnty!='US'].cnty.apply(lambda x: '{:05}'.format(x))
    tdf=tdf.merge(dictfips,on='cnty')
    stdf=tdf.groupby(['State_code','st_fips_code','step_ahead','horizon','fct_date'],as_index=False).sum().rename(columns={'st_fips_code':'cnty'})
    utdf=tdf.groupby(['step_ahead','horizon','fct_date'],as_index=False).sum()
    utdf.loc[utdf.index,'cnty']='US'
    tdf=tdf.append(stdf,ignore_index=True,sort=True)
    tdf=tdf.append(utdf,ignore_index=True,sort=True)
#     col=['cnty', 'step_ahead', 'horizon', 'fct_date', 'fct_mean', 'fct_lb', 'fct_ub']  
    if holieff==1:
        reddf=pd.read_csv('output/TG_reduction_factor_states.csv')
        reddf['fct_date']=tdf[tdf.fct_date<'2020-12-25']['fct_date'].max()
        reddf=reddf.merge(fipsdf[['FIPS','State_code']],on='State_code')
        reddf=reddf.rename(columns={'FIPS':'cnty'})
        tdf=tdf.merge(reddf,on=['cnty','fct_date','State_code'],how='left')
        tdf.loc[:,'red_fac']=tdf['red_fac'].fillna(0)
        tdf.loc[:,'fct_mean']=tdf['fct_mean']*(1-tdf['red_fac'])
        tdf.loc[:,'fct_lb']=tdf['fct_lb']*(1-tdf['red_fac'])
        tdf.loc[:,'fct_ub']=tdf['fct_ub']*(1-tdf['red_fac'])
    ind=tdf[tdf['fct_lb']<0].index
    tdf.loc[ind,'fct_lb']=0
    fdf=tdf[col]
    fdf=fdf.rename(columns={'b_fct_lb':'fct_lb','b_fct_ub':'fct_ub'})
    fdf=fdf[fdf.cnty.isin(locs)]
    fdf.loc[fdf['fct_lb']<=0,'fct_lb']=0
    fdf.loc[fdf['fct_ub']<=0,'fct_ub']=0
    fdf.loc[fdf.index,'fct_std']=(fdf['fct_ub']-fdf['fct_lb'])/3.96
    print(sdt,len(fdf[fdf.fct_std<0]['cnty'].unique()))
    fdf.to_csv('/project/biocomplexity/aniadiga/Forecasting/covid19_ensemble/output/ensemble/BMA/{}_BMA_ensemble.csv'.format(sdt),index=None)
    ll.append(sdt)
#     except:
#         continue

2021-04-19 0


In [13]:
# with open('output/submission_date_list.txt','w') as f:
#     for ll in dt_list:
#         f.writelines('{}\n'.format(ll))

In [14]:
# # fname='/project/biocomplexity/aniadiga/Forecasting/covid19_ensemble/output/ensemble/BMA/2020-11-16_BMA_ensemble.csv'
# %run quantile_conversion.py '/project/biocomplexity/aniadiga/Forecasting/covid19_ensemble/output/ensemble/BMA/2021-02-22_BMA_ensemble.csv' '2021-02-22' '/project/biocomplexity/aniadiga/Forecasting/covid19_ensemble/output/forecasthub/2021-02-22-UVA-Ensemble.csv'


In [15]:
%%bash -s "$sdt"
echo $1
python quantile_conversion.py /project/biocomplexity/aniadiga/Forecasting/covid19_ensemble/output/ensemble/BMA/$1_BMA_ensemble.csv $1 /project/biocomplexity/aniadiga/Forecasting/covid19_ensemble/output/forecasthub/$1-UVA-Ensemble.csv


2021-04-19


In [16]:
%%bash -s "$sdt"
ls /project/biocomplexity/aniadiga/Forecasting/covid19_ensemble/output/forecasthub/$1-UVA-Ensemble.csv

/project/biocomplexity/aniadiga/Forecasting/covid19_ensemble/output/forecasthub/2021-04-19-UVA-Ensemble.csv


In [17]:
df=pd.read_csv('/project/biocomplexity/aniadiga/Forecasting/covid19_ensemble/output/forecasthub/{}-UVA-Ensemble.csv'.format(sdt))

In [18]:
popdf=pd.read_csv('/project/biocomplexity/aniadiga/covid19-forecast-hub/data-locations/locations.csv')
df=df.merge(popdf,on='location',how='left')

In [19]:
locs=df[(df.value>df.population)]['location'].unique()

In [20]:
locs

array([], dtype=object)

In [21]:

df[(df.location.isin(locs))]

,forecast_date,target,target_end_date,location,type,quantile,value,abbreviation,location_name,population


In [22]:
# df=df[['forecast_date', 'target', 'target_end_date', 'location', 'type',
#        'quantile', 'value',]]
# df.to_csv('/project/biocomplexity/aniadiga/Forecasting/covid19_ensemble/output/forecasthub/2020-12-28-UVA-Ensemble.csv',index=None)

In [23]:
# df=pd.read_csv('output/test.csv'
(df[df.type=='quantile'].groupby(['location','target'],as_index=False)['value'].diff().fillna(0)>=0).all()

True

In [24]:
(df.value<df.population).all()

True

In [31]:
# l1=[]
# for loc in df.location.unique():
#     for stp in df.target.unique():
#         if not (df[(df.location==loc)&(df.type=='quantile')&(df.target==stp)]['value'].diff().fillna(0)>=0).all():
#             l1.append(loc)

In [15]:
outdir='/project/biocomplexity/aniadiga/Forecasting/covid19_ensemble/output/forecasthub/'
for fl in os.listdir(outdir):
    df=pd.read_csv(outdir+fl)
    temp=(df[df.type=='quantile'].groupby(['location','target'],as_index=False)['value'].diff().fillna(0)>=0).all()
    print(fl,temp)

2020-10-26-UVA-Ensemble.csv True
2020-10-12-UVA-Ensemble.csv True
2020-11-16-UVA-Ensemble.csv True
2020-07-06-UVA-Ensemble.csv True
2020-11-23-UVA-Ensemble.csv True
2020-08-03-UVA-Ensemble.csv True
2020-12-07-UVA-Ensemble.csv True
2020-08-31-UVA-Ensemble.csv True
2020-09-21-UVA-Ensemble.csv True
2020-06-08-UVA-Ensemble.csv True
2020-08-24-UVA-Ensemble.csv True
2020-11-30-UVA-Ensemble.csv True
2020-11-09-UVA-Ensemble.csv True
2020-07-20-UVA-Ensemble.csv True
2020-11-02-UVA-Ensemble.csv True
2020-08-10-UVA-Ensemble.csv True
2020-09-28-UVA-Ensemble.csv True
2020-10-19-UVA-Ensemble.csv True
2020-09-07-UVA-Ensemble.csv True
2020-06-15-UVA-Ensemble.csv True
2020-07-27-UVA-Ensemble.csv True
2020-08-17-UVA-Ensemble.csv True
2020-07-13-UVA-Ensemble.csv True
2020-06-22-UVA-Ensemble.csv True
2020-06-29-UVA-Ensemble.csv True
2020-09-14-UVA-Ensemble.csv True
2020-10-05-UVA-Ensemble.csv True


In [50]:
df=pd.read_csv('output/BMA_gl_{}.csv'.format(hrzndt))

In [53]:
df.method.unique()

array(['BMA-ensemble'], dtype=object)

In [54]:
%%bash -s "$sdt" "$hrzndt"
echo $1
python gl_quantile_conversion.py output/BMA_gl_$2.csv $1 output/$1-UVA-Ensemble.csv


2021-04-19


In [55]:
%%bash -s "$sdt"
echo $1
ls output/$1-UVA-Ensemble.csv


2021-04-19
output/2021-04-19-UVA-Ensemble.csv


In [56]:
df=pd.read_csv('output/{}-UVA-Ensemble.csv'.format(sdt))

In [57]:
# df['diff_val']=df[(df.location.isin(['DE','PL','GB']))&(df.type=='quantile')].groupby(['location','target','target_end_date'],as_index=False)[['value']].diff()

In [60]:
# df=df[~df[['forecast_date', 'target', 'target_end_date', 'location', 'type','quantile']].duplicated()]

In [61]:
(df[df.type=='quantile'].groupby(['location','target'],as_index=False)['value'].diff().fillna(0)>=0).all()

True

In [34]:
df.to_csv('output/{}-UVA-Ensemble.csv'.format(sdt),index=None)

In [47]:
# ind=
df['diff_val']=(df[df.type=='quantile'].groupby(['location','target'],as_index=False)[['value']].diff().fillna(0))


In [35]:
df

,forecast_date,target,target_end_date,location,type,quantile,value
0,2021-04-19,1 wk ahead inc case,2021-04-24,AT,point,NaN,22708.147121
1,2021-04-19,1 wk ahead inc case,2021-04-24,AT,quantile,0.025,15719.688409
2,2021-04-19,1 wk ahead inc case,2021-04-24,AT,quantile,0.100,18138.639582
3,2021-04-19,1 wk ahead inc case,2021-04-24,AT,quantile,0.250,20303.182638
4,2021-04-19,1 wk ahead inc case,2021-04-24,AT,quantile,0.500,22708.147121
5,2021-04-19,1 wk ahead inc case,2021-04-24,AT,quantile,0.750,25113.111604
6,2021-04-19,1 wk ahead inc case,2021-04-24,AT,quantile,0.900,27277.654660
7,2021-04-19,1 wk ahead inc case,2021-04-24,AT,quantile,0.975,29696.605834
8,2021-04-19,1 wk ahead inc case,2021-04-24,BE,point,NaN,18747.944448
9,2021-04-19,1 wk ahead inc case,2021-04-24,BE,quantile,0.025,18379.731836


In [ ]:
c(0.01, 0.025, seq(0.05, 0.95, by = 0.05), 0.975, 0.99)

In [79]:
x=np.array([0.01, 0.025, np.arange(0.05,0.95,0.05),.975,.99])

In [ ]:
x=np.append(np.array([0.01, 0.025]),np.arange(0.05,0.955,0.05))
np.append(x,np.array([0.975, 0.99]))

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [13]:
incub_list,recov_list,delay_list,dsgn_dates,scen_list

([3.0, 5.0, 7.0, 9.0],
 [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0],
 [7.0],
 ['2020-08-11',
  '2020-08-18',
  '2020-08-25',
  '2020-09-01',
  '2020-09-08',
  '2020-09-15',
  '2020-09-22',
  '2020-09-29',
  '2020-10-06',
  '2020-10-13',
  '2020-10-20',
  '2020-10-27',
  '2020-11-03',
  '2020-11-10'],
 [1.0, 0.5, 2.0])

In [60]:
indir='../output/design/'

In [59]:
64*12*3

2304

In [16]:
sdf=pd.DataFrame()
i=0
for obsdt in dsgn_dates:
    for scen in scen_list:
        for incub in incub_list:
            for recov in recov_list:
                for delay in delay_list:
                    i+=1
#                     sdf=sdf.append(pd.read_csv'../output/design/univ_pred_{}_{}_{}_{}_{}_{}.csv'.format('UVA', obsdt, scen, incub, recov, delay),sort=True)

In [20]:
pwd

'/sfs/qumulo/qproject/biocomplexity/aniadiga/gits/google_mobility_v2/regression'